In [1]:
import sys
sys.path.append('../')
import torch
import numpy as np
import pandas as pd

from pathlib import Path
from BaseTester import BaseTester
from demo import u0, f0
from MyPlot import *
from utils import *
import torch.nn.functional as F

In [ ]:
def kappa_func(xx, yy, mu):
	return np.exp(-mu * (xx**2 + yy**2))
GridSize = 256
area=((-1, -1), (1, 1))
(left, bottom), (right, top) = area
h = (right - left) / (GridSize - 1)

xx, yy = np.meshgrid(
	np.linspace(left, right, GridSize), np.linspace(bottom, top, GridSize)
)

mus = np.linspace(0, 1, 100)
force = np.ones((GridSize, GridSize))
# force = normal(xx, yy, h, [(right-left)/2, (top - bottom)/2])

sols = []
kappas = []
for mu in mus:
	kappa = kappa_func(xx, yy, mu) 
	A = reaction_A(GridSize, kappa).tocsr()
	b = reaction_b_dir(force, 0, h)
	ans = spsolve(A, b).reshape(GridSize, GridSize)
	sols.append(ans)
	kappas.append(kappa)

sols = np.stack(sols, axis=0)
kappas = np.stack(kappas, axis=0)

np.save('DLData/256/Demo2/ValSol.npy', sols)
np.save('DLData/256/Demo2/ValKappa.npy', kappas)
np.save('DLData/256/Demo2/ValMu.npy', mus)



In [5]:
class ResidualLoss(torch.nn.Module):
	def __init__(self, dtype, device, h, loss_fn, gd=0):
		super().__init__()
		self.dtype = dtype
		self.device = device
		self.h = h
		self.loss_fn = loss_fn
		self.gd = gd

		self.k = self._get_kernel([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
		self.hard_encode = lambda x: hard_encode(x, self.gd)

	def _get_kernel(self, k):
		k = torch.tensor(k, requires_grad=False)
		k = k.view(1, 1, 3, 3).repeat(1, 1, 1, 1).to(self.dtype).to(self.device)
		return k
		
	def forward(self, pre, f, ans):
		u = self.hard_encode(pre)
		x = F.conv2d(u, self.k)
		y = F.conv2d(ans[None, None, ...], self.k) 
		return torch.abs(y.squeeze() - x.squeeze())
	
class Demo1Tester(BaseTester):
	def __init__(self, k, **kwargs):
		super().__init__(**kwargs)
		self.img_save_path = kwargs["img_save_path"]
		self.k = k
		self.mesh()
		self.force = f0(self.xx, self.yy, k)					
		self.ans = u0(self.xx, self.yy, k)

		self.data = torch.from_numpy(np.stack([self.xx, self.yy])).to(self.dtype).to(self.device)[None, ...]
		
	def l2(self, pre, ans):
		diff = (pre - ans) ** 2 * self.h**2
		l2_errors = np.sqrt(np.sum(diff))
		return l2_errors.mean() 
	
	def mesh(self):
		(self.left, self.bottom), (self.right, self.top) = self.area
		self.dx = (self.right - self.left) / (self.GridSize - 1)
		self.dy = (self.top - self.bottom) / (self.GridSize - 1)
		self.h = self.dx

		self.xx, self.yy = np.meshgrid(
			np.linspace(self.left, self.right, self.GridSize),
			np.linspace(self.bottom, self.top, self.GridSize),
		)
	
	def test(self, exp_names, best_or_last):
		df = {
			'exp_name': [],
			'l2': []
		}

		for exp_name in exp_names:
			self.load_kwargs(exp_name)
			self.load_ckpt(best_or_last, exp_name)

			ls = exp_name.split('-')
			if 'MatRes' in exp_name or 'EnergyRes' in exp_name:
				act, method = ls[5], ls[-2]
				name = f'res_{act}'
			else:
				act, method = ls[5], ls[-3]
				name = f'{method.lower()}_{act}'

			with torch.no_grad():
				pre = self.net(self.data)
				pre = hard_encode(pre, 0).squeeze()
				pre = pre.cpu().numpy()

				ans = self.ans
				img_save_path = Path(f"{self.img_save_path}/{exp_name}")
				if not img_save_path.is_dir():
					img_save_path.mkdir(parents=True)
				self.save_img(f"{img_save_path}", pre, ans, name)

				l2_error = self.l2(pre.squeeze(), ans.squeeze())
				print(f"{self.k}\t&\t{self.kwargs['method']} \t&\t{self.kwargs['net_kwargs']['act']} \t&\t {self.kwargs['net_kwargs']['norm_method']} \t&\t{l2_error:.5e}")
				df['exp_name'].append(exp_name)
				df['l2'].append(l2_error)
		
		df = pd.DataFrame(df)
		df.to_csv(f"{self.img_save_path}/k={self.k}-l2.csv", index=False)
	
	def save_img(self, save_path, pre, ans, name):
		save_ctf(save_path, pre, ans, self.xx, self.yy, name = name)
		levels = np.linspace(self.ans.min(), self.ans.max(), 12)[2:-2]
		save_contour(save_path, pre, ans, self.xx, self.yy, levels=levels)

		bar_ticks = np.linspace(ans.min(), ans.max(), 10)
		save_surf(save_path, pre, self.xx, self.yy, f'surf_{name}')
		save_surf(save_path, ans, self.xx, self.yy, 'surf_ans')
	



In [6]:
GridSize = 256
for k in [1, 2, 4, 8]:
# k = 2
	Tester = Demo1Tester(
		k = k,
		GridSize=GridSize,
		area=((-1, -1), (1, 1)),
		ckpt_save_path=f'model_save',
		hyper_parameters_save_path = f'hyper_parameters', 
		img_save_path = f'./images/k={k}', 
		device='cuda',
		dtype=torch.float,
	)
	exp_names = []
	for p in Path(f'model_save/{GridSize}').iterdir():
		if 'Energy' in p.stem or 'Data' in p.stem or 'GradNorm' in p.stem:
			continue
		if f'k={k}' in p.stem and 'layer' in p.stem and 'Jac' in p.stem:
			exp_names.append(p.stem)
	Tester.test(
		exp_names,
		'last',
	)

1	&	Jac-3 	&	relu 	&	 layer 	&	1.84138e-04
1	&	Jac-3 	&	tanh 	&	 layer 	&	9.84459e-05
2	&	Jac-3 	&	tanh 	&	 layer 	&	2.72589e-04
2	&	Jac-3 	&	relu 	&	 layer 	&	3.37252e-04
4	&	Jac-3 	&	tanh 	&	 layer 	&	8.98275e-04
4	&	Jac-3 	&	relu 	&	 layer 	&	1.08090e-03
8	&	Jac-3 	&	relu 	&	 layer 	&	3.40284e-03
8	&	Jac-3 	&	tanh 	&	 layer 	&	3.34091e-03


In [1]:
import sys
sys.path.append('../')
import torch
import numpy as np
import pandas as pd

from pathlib import Path
from BaseTester import BaseTester
from MyPlot import *
from utils import hard_encode
from utils import normal, reaction_A, reaction_b_dir
from scipy.sparse.linalg import spsolve

In [2]:
from tqdm import tqdm

class Demo2Tester(BaseTester):
	def __init__(self, testN, **kwargs):
		super().__init__(**kwargs)
		self.testN = testN
		self.mesh()
		# self.force =  normal(self.xx, self.yy, self.h, ((self.right - self.left)/2, (self.top-self.bottom) / 2))
		self.force = np.ones((self.GridSize, self.GridSize))
		
		
	def l2(self, pre, ans):
		diff = (pre - ans) ** 2 * self.h**2
		l2_errors = np.sqrt(np.sum(diff))
		return l2_errors.mean() 
	
	def mesh(self):
		(self.left, self.bottom), (self.right, self.top) = self.area
		self.dx = (self.right - self.left) / (self.GridSize - 1)
		self.dy = (self.top - self.bottom) / (self.GridSize - 1)
		self.h = self.dx

		self.xx, self.yy = np.meshgrid(
			np.linspace(self.left, self.right, self.GridSize),
			np.linspace(self.bottom, self.top, self.GridSize),
		)
	
	def kappa(self, mu):
		return np.exp(-mu * (self.xx ** 2 + self.yy**2))
	
	def test(self, exp_names, best_or_last, kappas=None):
		if kappas is None:
			mus = np.random.uniform(0, 1,  self.testN)
			print(mus)
			kappas = np.stack([self.kappa(mu)[np.newaxis, ...] for mu in mus ])
		ans = []
		b = reaction_b_dir(self.force, 0, self.h)
		testN = len(kappas)
		for i in tqdm(range(testN)):
			k = kappas[i].squeeze()
			A = reaction_A(self.GridSize, k).tocsr()
			ans.append(spsolve(A, b).reshape( self.GridSize, self.GridSize))
		ans = np.stack(ans)

		# np.save('./Demo2-TestAns.npy', ans)
		# np.save('./Demo2-TestData.npy', kappas)
		
		# kappas = np.load('./Demo2-TestData.npy')
		# ans = np.load('./Demo2-TestAns.npy')[:, np.newaxis, ...]
		# testN = len(kappas)

		kappas = kappas[:self.testN]
		ans = ans[:self.testN]

		data = torch.from_numpy(kappas).to(self.dtype).to(self.device)
		for exp_name in exp_names:
			df = {
				# 'test_case': [],
				'l2': []
			}
		
			self.load_kwargs(exp_name)
			self.load_ckpt(best_or_last, exp_name)
		
			with torch.no_grad():
				pre = self.net(data)
				pre = hard_encode(pre, 0)
				pre = pre.cpu().numpy().squeeze()

			print(pre.shape)
			save_path = Path(f"{self.img_save_path}/{exp_name}")
			if not save_path.is_dir():	save_path.mkdir(parents=True)

			l2_errors = []
			for i in range(self.testN):
				p = save_path/f"Case-{i}"
				if not p.is_dir():	
					p.mkdir(parents=True)
				self.save_img(p, pre[i].squeeze(), ans[i].squeeze(), kappas[i].squeeze())
				l2_error = self.l2(pre[i], ans[i])
				l2_errors.append(l2_error)
				# df['test_case'].append(i)
				df['l2'].append(l2_error)
			df = pd.DataFrame(df)
			df.to_csv(f"{save_path}.csv")

			method, k, trainN = exp_name.split('-')[-3:]
			print(f"{method}\t&\t{k}\t&\t{trainN}\t&\t{np.array(l2_errors).mean()}")
	
	def save_img(self, save_path, pre, ans, cof):
		save_img_force(save_path, cof, name='cof', vmin=0.1, vmax=2)
		save_ctf(save_path, pre, ans, self.xx, self.yy)
		levels = np.linspace(ans.min(), ans.max(), 10)[2:-1]
		save_contour(save_path, pre, ans, self.xx, self.yy, levels=levels)
		save_surf(save_path, pre, self.xx, self.yy, 'surf_pre')
		save_surf(save_path, ans, self.xx, self.yy, 'surf_ans')
	

In [3]:
ans = np.load('./Demo2-TestData.npy')
a = ans[0]
print(ans.shape)
# xx, yy = np.meshgrid(
#     np.linspace(-1, 1, 256),
#     np.linspace(-1, 1, 256)
# )
# plt.contourf(xx, yy, a)
# plt.imshow(a)

(1000, 1, 256, 256)


In [4]:
GridSize = 256
Tester = Demo2Tester(
	testN=3,
	GridSize=GridSize,
	area=((-1, -1), (1, 1)),
	ckpt_save_path=f'model_save',
	hyper_parameters_save_path = f'hyper_parameters', 
	img_save_path = f'./images/Demo2', 
	device='cuda',
	dtype=torch.float,
)
exp_names = []
for p in Path(f'model_save/{GridSize}').iterdir():
	if 'Demo2' in p.stem and '25-5000' in p.stem:
		exp_names.append(p.stem)
		# print(p.stem.split('-')[-3:])
Tester.test(
	exp_names,
	'last',
	# kappas=np.ones((0.1, 1, 256, 256)) * 0.2

)

[0.54813546 0.22706493 0.33733692]


100%|██████████| 3/3 [00:02<00:00,  1.37it/s]


(3, 256, 256)
Desc	&	25	&	5000	&	7.609739551948419e-05
(3, 256, 256)
Jac	&	25	&	5000	&	0.0001515646022791218


In [ ]:
Jac		&	1	&	100		&	0.10053069604839905
Jac		&	1	&	500		&	0.009142189845182969
Jac		&	1	&	5000	&	0.00037016732885555094

Jac		&	5	&	100		&	0.09415204063292569
Jac		&	5	&	500		&	0.0011334540588817238
Jac		&	5	&	5000	&	0.00022657807895347414

Jac		&	25	&	100		&	0.017286803618664685
Jac		&	25	&	500		&	0.0006991993553601014
Jac		&	25	&	5000	&	0.00022067673407101936

Desc	&	1	&	100	`	&	0.09763871757550524
Desc	&	1	&	500		&	0.0028756982893573682
Desc	&	1	&	5000	&	0.0005872470832737366

Desc	&	5	&	100		&	0.0528270640213064
Desc	&	5	&	500		&	0.005454670240321949
Desc	&	5	&	5000	&	0.000374092234718699

Desc	&	25	&	100	`	&	0.006820678593188237
Desc	&	25	&	500		&	0.0005588491615975406
Desc	&	25	&	5000	&	0.00010590824877410589


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

p = Path('./images/Demo2/256')

method = 'Desc'
for csv in p.glob(f'*{method}*.csv'):
	K, M = csv.stem.split('-')[-2:]
	df = pd.read_csv(csv)
	mean = df['l2'].values.mean()
	print(f"{method}\t&\t{K}\t&\t{M}\t&\t{mean}")
